In [2]:
import pandas as pd
WorldGDP = "../Outputs/WorldGDP.csv"
WorldGDPRaw = pd.read_csv(WorldGDP, encoding="ISO-8859-1")



#eliminate nulls
# WorldGDPRaw["nullG"]=pd.isna(WorldGDPRaw['GDP'])
# WorldGDPDF=WorldGDPRaw.loc[WorldGDPRaw["nullG"]==False]

WorldGDPRaw = WorldGDPRaw.dropna(how='any')
WorldGDPRaw.head()

,country,countryCode,GDP,POP
0,Afghanistan,AF,550.068459,35530081.0
1,Albania,AL,4537.579056,2873457.0
2,Algeria,DZ,4055.247211,41318142.0
3,American Samoa,AS,11394.475297,55641.0
4,Andorra,AD,39146.548836,76965.0


In [3]:
Wine = "../Resources/winedata.csv"
WineDF = pd.read_csv(Wine, encoding="ISO-8859-1")
WineDF.reset_index(inplace=True)
WineDF = WineDF.dropna(how='any')
WineDF.head()

,index,country,points,price,province,taster_name,taster_twitter_handle,title,variety
0,0,Portugal,87,15.0,Douro,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red
1,1,US,87,14.0,Oregon,Paul Gregutt,@paulgwineÂ,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris
2,2,US,87,65.0,Oregon,Paul Gregutt,@paulgwineÂ,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir
3,3,Spain,87,15.0,Northern Spain,Michael Schachner,@wineschach,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot
4,4,Italy,87,16.0,Sicily & Sardinia,Kerin OâKeefe,@kerinokeefe,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato


In [4]:

GroupByDF = WineDF.groupby(['country']) #this only indicates what to group by we have to select after
distinctCountry=GroupByDF.count() # returns the mean of all columns grouped by the column we chose. we have to do this to have a new dataframe to perform calcs
distinctCountry.reset_index(inplace=True)

WineCountryDF=distinctCountry[["country"]]
WineCountryDF.head()

,country
0,Argentina
1,Armenia
2,Australia
3,Austria
4,Bosnia and Herzegovina


In [5]:
# left join
CountryTable = pd.merge(WineCountryDF, WorldGDPRaw, on="country", how="left")
CountryTable.reset_index(inplace=True)
CountryTable["countryId"] = CountryTable["index"]+1
CountryTable.head()

,index,country,countryCode,GDP,POP,countryId
0,0,Argentina,AR,14398.358771,44271041.0,1
1,1,Armenia,AM,3936.798320,2930450.0,2
2,2,Australia,AU,53793.537256,24601860.0,3
3,3,Austria,AT,47380.829637,8797566.0,4
4,4,Bosnia and Herzegovina,BA,5148.208517,3507017.0,5


In [6]:
CountryTable.to_csv("../Outputs/CountryTable.csv", index=False)

Insert into My sql database

In [7]:
import pandas as pd
ImportTable = "../Outputs/CountryTable.csv"
CountryTable = pd.read_csv(ImportTable, encoding="ISO-8859-1")

CountryToExport=CountryTable[["countryId","country","countryCode","GDP","POP"]]
CountryToExport.head()

,countryId,country,countryCode,GDP,POP
0,1,Argentina,AR,14398.358771,44271041.0
1,2,Armenia,AM,3936.798320,2930450.0
2,3,Australia,AU,53793.537256,24601860.0
3,4,Austria,AT,47380.829637,8797566.0
4,5,Bosnia and Herzegovina,BA,5148.208517,3507017.0


In [8]:
from sqlalchemy import create_engine

In [9]:

rds_connection_string = "root:Sanangel1@127.0.0.1/Wine_Ratings"
engine = create_engine(f'mysql+pymysql://{rds_connection_string}')


In [10]:
engine.table_names()

['country', 'state', 'taster', 'wine']

In [18]:
CountryToExport.to_sql(name='country', con=engine, if_exists='append', index=False)

In [19]:
pd.read_sql_query('select * from country', con=engine).head()

,countryId,country,countryCode,GDP,POP
0,1,Argentina,AR,14398.36,44271041.0
1,2,Armenia,AM,3936.80,2930450.0
2,3,Australia,AU,53793.54,24601860.0
3,4,Austria,AT,47380.83,8797566.0
4,5,Bosnia and Herzegovina,BA,5148.21,3507017.0
